In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data directory and parameters
data_dir = "../sketches"  
target_size = (128, 128)
batch_size = 64

# Create an ImageDataGenerator instance with the desired preprocessing and augmentation.
# Here, we'll use the custom preprocessing_function from earlier to do resizing, inversion, and normalization.
def preprocessing_function(img):
    # For an input image (as a NumPy array) with shape (H, W, 3) or (H, W),
    # convert to grayscale (if needed), resize to 128x128 using bilinear interpolation,
    # invert pixel values so that zeros correspond to background (white), and rescale to [0,1]
    import cv2
    if img.ndim == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
    img_inverted = 255 - img_resized  # invert image
    img_norm = img_inverted.astype("float32") / 255.0
    # Expand dims to create shape (128,128,1)
    return np.expand_dims(img_norm, axis=-1)

datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    horizontal_flip=True,
    validation_split=0.2  # if you also want a validation split
)

# Create a generator that reads from the data directory.
# Here, "categorical" mode assumes that each subfolder is a class and creates one-hot labels.
generator = datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Option 1: Save the raw processed images and labels

# Get the number of samples
num_samples = generator.samples
print("Total samples:", num_samples)

# Preallocate arrays for images and labels
# Images will be in shape (num_samples, 128, 128, 1)
X = np.zeros((num_samples, target_size[0], target_size[1], 1), dtype="float32")
y = np.zeros((num_samples, len(generator.class_indices)), dtype="float32")

# Iterate over the generator and collect the data
i = 0
for batch in generator:
    batch_x, batch_y = batch  # unpack images and labels
    batch_size_actual = batch_x.shape[0]
    X[i:i+batch_size_actual] = batch_x
    y[i:i+batch_size_actual] = batch_y
    i += batch_size_actual
    if i >= num_samples:
        break

# Save the arrays to disk for easy access (others can load these without rerunning augmentation)
np.save("preprocessed_images.npy", X)
np.save("preprocessed_labels.npy", y)
print("Saved preprocessed images and labels to preprocessed_images.npy and preprocessed_labels.npy")

# Option 2: You may also want to save the generator's class indices mapping for reference:
np.save("class_indices.npy", generator.class_indices)
print("Saved class indices mapping to class_indices.npy")

2025-04-09 11:22:23.506798: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 11:22:23.759521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744177943.872681    7392 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744177943.904297    7392 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744177944.107649    7392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Found 1200 images belonging to 15 classes.
Total samples: 1200
Saved preprocessed images and labels to preprocessed_images.npy and preprocessed_labels.npy
Saved class indices mapping to class_indices.npy


In [2]:
X = np.load("preprocessed_images.npy")
y = np.load("preprocessed_labels.npy")
class_indices = np.load("class_indices.npy", allow_pickle=True).item()